In [1]:
import pandas as pd
import numpy as np
from cataclop.ml.pipeline import factories
from cataclop.ml.exploration import random_race
from cataclop.ml import preprocessing

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

NAN_FLAG = 10000

In [2]:
#d = factories.Dataset.factory('default', {"from": '2013-05-10'})
d = factories.Dataset.factory('default', {"from": '2018-05-10'})
d.load(force=False)


In [ ]:
#d.save()

In [ ]:
d.players.head()

In [3]:
m = factories.Model.factory('default')

In [4]:
m.load()

In [ ]:
df = m.train(d)

In [ ]:
random_race(df, cols=['position', 'sub_category', 'num', 'final_odds', 'final_odds_ref', 'odds_0', 'odds_1', 'odds_2'] + d.agg_features, n=1)

In [8]:
random_race(df, ['sub_category', 'num', 'position', 'final_odds', 'pred_rf_10_1', 'pred_rf_100_1'])

sub_category  num  position  final_odds  pred_rf_10_1  \
race_id                                                              
19709   6408     GROUPE_I    1       2.0        43.1         4.995   
        6409     GROUPE_I    2       6.0        22.6         4.975   
        6410     GROUPE_I    3       5.0        10.0         5.020   
        6411     GROUPE_I    4       9.0        38.4         4.560   
        6412     GROUPE_I    5      11.0        22.1         3.910   
        6413     GROUPE_I    6       1.0        17.0         6.780   
        6414     GROUPE_I    7       3.0         7.7         3.620   
        6415     GROUPE_I    8       4.0         1.6         3.830   
        6416     GROUPE_I    9      10.0        61.3         3.655   
        6417     GROUPE_I   10       8.0        57.3         5.130   
        6418     GROUPE_I   11       7.0        10.8         3.910   
        6419     GROUPE_I   12      12.0       130.1         4.790   

              pred_rf_100_1  
race_id                      
19709   6408       3.416833  
        6409       4.611000  
        6410       4.792500  
        6411       4.633000  
        6412       3.215833  
        6413       7.179500  
        6414       4.751833  
        6415       2.997833  
        6416       4.302833  
        6417       4.030000  
        6418       3.215833  
        6419       5.381000

In [ ]:
for model in m.models:
    clf = model['estimators'][-1]['pipeline'].steps[-1][1]
    if hasattr(clf, 'feature_importances_'):
        fi = pd.Series(clf.feature_importances_, index=m.features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
        print(fi.sort_values(ascending=False))
        break

In [9]:
targets = ['pred_knn_10_1', 'pred_knn_30_1', 'pred_rf_10_1', 'pred_rf_100_1', 'pred_mlp_1']

df[targets].describe()

,pred_knn_10_1,pred_knn_30_1,pred_rf_10_1,pred_rf_100_1,pred_mlp_1
count,11637.000000,11637.000000,11637.000000,11637.000000,11637.000000
mean,3.868954,3.994431,4.819002,4.935321,4.295930
std,1.499579,1.079434,2.891673,2.643912,2.004323
min,0.000000,0.840000,0.130000,0.432000,-2.409320
25%,2.845000,3.216667,2.825000,3.215000,2.864884
50%,3.685000,4.018333,4.250000,4.404500,4.191871
75%,4.750000,4.780000,6.135000,6.052500,5.649845
max,11.980000,7.990000,31.270000,33.716750,17.087822


In [10]:
races = df.sort_values('start_at').groupby('race_id')

bets = []

stash = 500

for (id, race) in races:
    
    candidate_bets = []
    
    nums = []
    
    for target in targets:

        r = race.sort_values(by=target, ascending=False)

        odds = r['final_odds_ref'].min()
        nth = len(r[r['final_odds_ref']<odds])+1

        N = 1

        for n in range(N):

            if len(r) <= N:
                continue

            player = r.iloc[n]

            if player['num'] in nums:
                continue

            odds = player['final_odds_ref']

            if odds > 20:
                break

            nth = (r['final_odds_ref']<odds).sum()+1

            bet = 1.

            profit = player['winner_dividend']/100.0 * bet - bet

            stash = stash + profit
            
            row = [id, player['date'], player['num'], odds, player['final_odds'], target, player[target], bet, profit]
            
            for f in m.features:
                row.append(player[f])
            for f in m.categorical_features:
                row.append(player[f])

            candidate_bets.append( row )

            nums.append(player['num'])
            
            break
    
    if len(candidate_bets) == 1:
        bets += candidate_bets
        
        
cols = ['id', 'date', 'num', 'odds_ref', 'odds_final', 'target', 'pred', 'bet', 'profit'] + m.features + m.categorical_features

bets = pd.DataFrame(bets, columns=cols)

bets.index = bets['date']

bets = bets.sort_index()

bets['bets'] = bets['bet'].cumsum()
bets['stash'] = bets['profit'].cumsum()

In [16]:
bets[(bets['target'] == 'pred_rf_100_1') & (bets['odds_ref'] > 1) & (bets['category'] != 'PfLAT')][['num', 'odds_ref', 'odds_final', 'pred', 'profit']]

,num,odds_ref,odds_final,pred,profit
date,,,,,
2018-05-18,7,20.0,30.5,6.373000,-1.0
2018-05-20,2,8.3,8.1,4.845250,-1.0
2018-05-21,3,11.4,8.8,4.771000,10.3
2018-05-21,4,7.4,10.3,6.826533,-1.0
2018-05-30,11,8.5,6.7,10.530417,-1.0
2018-05-31,4,12.7,27.1,11.477500,-1.0
2018-06-04,12,6.2,5.1,10.359381,-1.0
2018-06-04,7,6.9,6.6,5.102000,-1.0
2018-06-06,6,17.5,22.9,7.561667,-1.0


In [ ]:
bets

In [19]:
bets[(bets['target'] == 'pred_knn_30_1') & (bets['odds_ref'] > 1) & (bets['category'] != 'PMLAT')]['profit'].describe()

count    66.000000
mean     -0.063636
std       2.265367
min      -1.000000
25%      -1.000000
50%      -1.000000
75%      -1.000000
max       8.700000
Name: profit, dtype: float64

In [17]:
bets[['num', 'target', 'odds_ref', 'odds_final', 'pred', 'profit']]

,num,target,odds_ref,odds_final,pred,profit
date,,,,,,
2018-05-10,2,pred_knn_10_1,15.3,18.3,5.950000,-1.0
2018-05-10,9,pred_rf_10_1,11.7,13.7,4.555000,-1.0
2018-05-10,7,pred_knn_10_1,8.6,10.6,8.105000,-1.0
2018-05-10,14,pred_knn_10_1,16.4,27.8,8.490000,-1.0
2018-05-10,2,pred_knn_30_1,5.2,4.7,4.881667,-1.0
2018-05-11,2,pred_rf_10_1,11.8,12.6,7.860000,-1.0
2018-05-11,14,pred_knn_10_1,17.0,17.0,7.680000,-1.0
2018-05-11,2,pred_knn_30_1,12.7,16.7,4.958333,-1.0
2018-05-11,5,pred_knn_10_1,15.5,9.8,3.570000,-1.0


In [6]:
df = m.predict(d)

In [ ]:
df['pred_knn_10_1'].describe()

In [ ]:
m.save()

In [ ]:
m.load()